In [ ]:
# importing the libraries
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup
from wordcloud import STOPWORDS
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# open file and save the text in to a list
urlList = open('URLList.txt','r')
lines =  [line for line in urlList.readlines()]
urlList.close()

In [ ]:
%%time
# remove ', <, > in the out put .To remove them from output add them to STOPWORDS list.
STOPWORDS.add('<')
STOPWORDS.add('>')
STOPWORDS.add('\'')
STOPWORDS.add('\'\'')
STOPWORDS.add('h1')
STOPWORDS.add('h2')
STOPWORDS.add('h3')
STOPWORDS.add('h4')
STOPWORDS.add('h5')
STOPWORDS.add('span')
STOPWORDS.add('class=')

Wall time: 1 ms


In [ ]:
%%time
print("url list write-Begin")
#read each url and get the content
for line in lines:
    line=line.strip()

    if(line == ''):
        continue
    #make the request and check object type
    request1 = requests.get(line)

    type(request1)
    #extract html from response
    webPage1 = request1.text

    # Parse the html content
    soup = BeautifulSoup(webPage1, "lxml")

        #tokenize
        #extract common words.Filter keywords
        #stemming algorithm
        #set default weights for keywords

    def SetWeightKeyWords(x,weight):

        Stem_words = []
        ps =PorterStemmer()

        for tag in soup.find_all(x):

            token = word_tokenize(str(tag))
            for i in token:
                rootWord=ps.stem(i)
                Stem_words.append(rootWord)

        word_freq = pd.Series(Stem_words).value_counts(ascending=False)

        #Remove stopwords
        word_freq = word_freq.drop(labels=STOPWORDS, errors='ignore')
        #Create word frequencies using word_list
        word_freq_df           = pd.DataFrame(word_freq, columns=['wordAndFreq'])
        word_freq_df['freq']   = word_freq_df["wordAndFreq"].astype(str).str.split().str[0]
        word_freq_df['Weight'] = weight

        words=[]
        for i in word_freq_df.iterrows():
            words.append(i[0])

        word_freq_df['Word'] = words
        return word_freq_df.head(10)


    def CalCulateTotal():
        word_freq_df_Title = pd.DataFrame(SetWeightKeyWords('title',1))
        word_freq_df_h1    = pd.DataFrame(SetWeightKeyWords('h1',0.8))
        word_freq_df_h2    = pd.DataFrame(SetWeightKeyWords('h2',0.7))
        word_freq_df_h3    = pd.DataFrame(SetWeightKeyWords('h3',0.6))
        word_freq_df_h4    = pd.DataFrame(SetWeightKeyWords('h4',0.5))
        word_freq_df_h5    = pd.DataFrame(SetWeightKeyWords('h5',0.4))
        word_freq_df_p     = pd.DataFrame(SetWeightKeyWords('p',0.5))

        # write algorithm calculate weight for each page in descending order

        #Compare DF1 with DF2 and calc weight . title to para
        word_freq_df_Title["TotWeight"] = np.where(word_freq_df_Title['Word'].isin(word_freq_df_h1['Word']),1.8,1)
        word_freq_df_h1["TotWeight"]    = np.where(word_freq_df_h1['Word'].isin(word_freq_df_Title['Word']),1.8,0.8)

        #remove df1 words from df2
        indexNames = word_freq_df_h1[word_freq_df_h1["TotWeight"] > 0.8 ].index
        word_freq_df_h1.drop(indexNames , inplace=True)

        #Compare DF1 with DF3 and calc weight
        word_freq_df_Title["TotWeight"] = np.where(word_freq_df_Title['Word'].isin(word_freq_df_h2['Word']),word_freq_df_Title["TotWeight"]+0.7,word_freq_df_Title["TotWeight"])
        word_freq_df_h2["TotWeight"]    = np.where(word_freq_df_h2['Word'].isin(word_freq_df_Title['Word']),1.7,0.7)
        #remove df1 words from df3
        indexNames = word_freq_df_h2[word_freq_df_h2["TotWeight"] > 0.7 ].index
        word_freq_df_h2.drop(indexNames , inplace=True)

        #Compare DF1 with DF4 and calc weight assign it to DF4
        word_freq_df_Title["TotWeight"] = np.where(word_freq_df_Title['Word'].isin(word_freq_df_h3['Word']),word_freq_df_Title["TotWeight"]+0.6,word_freq_df_Title["TotWeight"])
        word_freq_df_h3["TotWeight"]    = np.where(word_freq_df_h3['Word'].isin(word_freq_df_Title['Word']),1.6,0.6)
        #remove df1 words from df4
        indexNames = word_freq_df_h3[word_freq_df_h3["TotWeight"] > 0.6 ].index
        word_freq_df_h3.drop(indexNames , inplace=True)

        #Compare DF1 with DF5 and calc weight
        word_freq_df_Title["TotWeight"] = np.where(word_freq_df_Title['Word'].isin(word_freq_df_h4['Word']),word_freq_df_Title["TotWeight"]+0.5,word_freq_df_Title["TotWeight"])
        word_freq_df_h4["TotWeight"]    = np.where(word_freq_df_h4['Word'].isin(word_freq_df_Title['Word']),1.5,0.5)
        #remove df1 words from df5
        indexNames = word_freq_df_h4[word_freq_df_h4["TotWeight"] > 0.5 ].index
        word_freq_df_h4.drop(indexNames , inplace=True)

        #Compare DF1 with DF6 and calc weight
        word_freq_df_Title["TotWeight"] = np.where(word_freq_df_Title['Word'].isin(word_freq_df_h5['Word']),word_freq_df_Title["TotWeight"]+0.4,word_freq_df_Title["TotWeight"])
        word_freq_df_h5["TotWeight"]    = np.where(word_freq_df_h5['Word'].isin(word_freq_df_Title['Word']),1.4,0.4)
        #remove df1 words from df6
        indexNames = word_freq_df_h5[word_freq_df_h5["TotWeight"] > 0.4 ].index
        word_freq_df_h5.drop(indexNames , inplace=True)

        #Compare DF1 with DF7 and calc weight
        word_freq_df_Title["TotWeight"] = np.where(word_freq_df_Title['Word'].isin(word_freq_df_p['Word']),word_freq_df_Title["TotWeight"]+0.5,word_freq_df_Title["TotWeight"])
        word_freq_df_p["TotWeight"]     = np.where(word_freq_df_p['Word'].isin(word_freq_df_Title['Word']),1.5,0.5)
        #remove df1 words from df7
        indexNames = word_freq_df_p[word_freq_df_p["TotWeight"] > 0.5 ].index
        word_freq_df_p.drop(indexNames , inplace=True)

        #D1- D7 comparison done

        #D2-D7 comparison
        word_freq_df_h1["TotWeight"] = np.where(word_freq_df_h1['Word'].isin(word_freq_df_h2['Word']),word_freq_df_h1["Weight"]+0.7,word_freq_df_h1["Weight"])
        word_freq_df_h2["TotWeight"] = np.where(word_freq_df_h2['Word'].isin(word_freq_df_h1['Word']),word_freq_df_h2["Weight"]+0.8,word_freq_df_h2["Weight"])
        #remove df2 words from df3
        indexNames = word_freq_df_h2[word_freq_df_h2["TotWeight"] > 0.7 ].index
        word_freq_df_h2.drop(indexNames , inplace=True)

        word_freq_df_h1["TotWeight"] = np.where(word_freq_df_h1['Word'].isin(word_freq_df_h3['Word']),word_freq_df_h1["TotWeight"]+0.6,word_freq_df_h1["TotWeight"])
        word_freq_df_h3["TotWeight"] = np.where(word_freq_df_h3['Word'].isin(word_freq_df_h1['Word']),word_freq_df_h3["Weight"]+0.8,word_freq_df_h3["Weight"])
        #remove df2 words from df3
        indexNames = word_freq_df_h3[word_freq_df_h3["TotWeight"] > 0.6 ].index
        word_freq_df_h3.drop(indexNames , inplace=True)

        word_freq_df_h1["TotWeight"] = np.where(word_freq_df_h1['Word'].isin(word_freq_df_h4['Word']),word_freq_df_h1["TotWeight"]+0.5,word_freq_df_h1["TotWeight"])
        word_freq_df_h4["TotWeight"] = np.where(word_freq_df_h4['Word'].isin(word_freq_df_h1['Word']),word_freq_df_h4["Weight"]+0.8,word_freq_df_h4["Weight"])
        indexNames = word_freq_df_h4[word_freq_df_h4["TotWeight"] > 0.5 ].index
        word_freq_df_h4.drop(indexNames , inplace=True)


        word_freq_df_h1["TotWeight"] = np.where(word_freq_df_h1['Word'].isin(word_freq_df_h5['Word']),word_freq_df_h1["TotWeight"]+0.4,word_freq_df_h1["TotWeight"])
        word_freq_df_h5["TotWeight"] = np.where(word_freq_df_h5['Word'].isin(word_freq_df_h1['Word']),word_freq_df_h5["Weight"]+0.8,word_freq_df_h5["Weight"])
        indexNames = word_freq_df_h5[word_freq_df_h5["TotWeight"] > 0.4 ].index
        word_freq_df_h5.drop(indexNames , inplace=True)


        word_freq_df_h1["TotWeight"] = np.where(word_freq_df_h1['Word'].isin(word_freq_df_p['Word']),word_freq_df_h1["TotWeight"]+0.5,word_freq_df_h1["TotWeight"])
        word_freq_df_p["TotWeight"] = np.where(word_freq_df_p['Word'].isin(word_freq_df_h1['Word']),word_freq_df_p["Weight"]+0.8,word_freq_df_p["Weight"])
        indexNames = word_freq_df_p[word_freq_df_p["TotWeight"] > 0.5 ].index
        word_freq_df_p.drop(indexNames , inplace=True)


        #D3-D7  comparison heading 2 paragraph
        word_freq_df_h2["TotWeight"] = np.where(word_freq_df_h2['Word'].isin(word_freq_df_h3['Word']),word_freq_df_h2["Weight"]+0.6,word_freq_df_h2["Weight"])
        word_freq_df_h3["TotWeight"] = np.where(word_freq_df_h3['Word'].isin(word_freq_df_h2['Word']),word_freq_df_h3["Weight"]+0.7,word_freq_df_h3["Weight"])
        indexNames = word_freq_df_h3[word_freq_df_h3["TotWeight"] > 0.6 ].index
        word_freq_df_h3.drop(indexNames , inplace=True)

        word_freq_df_h2["TotWeight"] = np.where(word_freq_df_h2['Word'].isin(word_freq_df_h4['Word']),word_freq_df_h2["TotWeight"]+0.5,word_freq_df_h2["TotWeight"])
        word_freq_df_h4["TotWeight"] = np.where(word_freq_df_h4['Word'].isin(word_freq_df_h2['Word']),word_freq_df_h4["Weight"]+0.7,word_freq_df_h4["Weight"])
        indexNames = word_freq_df_h4[word_freq_df_h4["TotWeight"] > 0.5 ].index
        word_freq_df_h4.drop(indexNames , inplace=True)


        word_freq_df_h2["TotWeight"] = np.where(word_freq_df_h2['Word'].isin(word_freq_df_h5['Word']),word_freq_df_h2["TotWeight"]+0.4,word_freq_df_h2["TotWeight"])
        word_freq_df_h5["TotWeight"] = np.where(word_freq_df_h5['Word'].isin(word_freq_df_h2['Word']),word_freq_df_h5["Weight"]+0.7,word_freq_df_h5["Weight"])
        indexNames = word_freq_df_h5[word_freq_df_h5["TotWeight"] > 0.4 ].index
        word_freq_df_h5.drop(indexNames , inplace=True)

        word_freq_df_h2["TotWeight"] = np.where(word_freq_df_h2['Word'].isin(word_freq_df_p['Word']),word_freq_df_h2["TotWeight"]+0.5,word_freq_df_h2["TotWeight"])
        word_freq_df_p["TotWeight"] = np.where(word_freq_df_p['Word'].isin(word_freq_df_h2['Word']),word_freq_df_p["Weight"]+0.7,word_freq_df_p["Weight"])
        indexNames = word_freq_df_p[word_freq_df_p["TotWeight"] > 0.5].index
        word_freq_df_p.drop(indexNames , inplace=True)

        #D4 -D7 comparison heading 3 to para
        word_freq_df_h3["TotWeight"] = np.where(word_freq_df_h3['Word'].isin(word_freq_df_h4['Word']),word_freq_df_h3["Weight"]+0.5,word_freq_df_h3["Weight"])
        #add 0.6
        word_freq_df_h4["TotWeight"] = np.where(word_freq_df_h4['Word'].isin(word_freq_df_h3['Word']),word_freq_df_h4["Weight"]+0.6,word_freq_df_h4["Weight"])
        indexNames = word_freq_df_h4[word_freq_df_h4["TotWeight"] > 0.5 ].index
        word_freq_df_h4.drop(indexNames , inplace=True)


        word_freq_df_h3["TotWeight"] = np.where(word_freq_df_h3['Word'].isin(word_freq_df_h5['Word']),word_freq_df_h3["TotWeight"]+0.4,word_freq_df_h3["TotWeight"])
        word_freq_df_h5["TotWeight"] = np.where(word_freq_df_h5['Word'].isin(word_freq_df_h3['Word']),word_freq_df_h5["Weight"]+0.6,word_freq_df_h5["Weight"])
        indexNames = word_freq_df_h5[word_freq_df_h5["TotWeight"] > 0.4 ].index
        word_freq_df_h5.drop(indexNames , inplace=True)

        word_freq_df_h3["TotWeight"] = np.where(word_freq_df_h3['Word'].isin(word_freq_df_p['Word']),word_freq_df_h3["TotWeight"]+0.5,word_freq_df_h3["TotWeight"])
        word_freq_df_p["TotWeight"]  = np.where(word_freq_df_p['Word'].isin(word_freq_df_h3['Word']),word_freq_df_p["Weight"]+0.6,word_freq_df_p["Weight"])
        indexNames = word_freq_df_p[word_freq_df_p["TotWeight"] > 0.5].index
        word_freq_df_p.drop(indexNames , inplace=True)

        #D5 -D7 comparison
        word_freq_df_h4["TotWeight"] = np.where(word_freq_df_h4['Word'].isin(word_freq_df_h5['Word']),word_freq_df_h4["Weight"]+0.4,word_freq_df_h4["Weight"])
        word_freq_df_h5["TotWeight"] = np.where(word_freq_df_h5['Word'].isin(word_freq_df_h4['Word']),word_freq_df_h5["Weight"]+0.5,word_freq_df_h5["Weight"])
        indexNames = word_freq_df_h5[word_freq_df_h5["TotWeight"] > 0.4 ].index
        word_freq_df_h5.drop(indexNames , inplace=True)

        word_freq_df_h4["TotWeight"] = np.where(word_freq_df_h4['Word'].isin(word_freq_df_p['Word']),word_freq_df_h4["TotWeight"]+0.5,word_freq_df_h4["TotWeight"])
        word_freq_df_p["TotWeight"]  = np.where(word_freq_df_p['Word'].isin(word_freq_df_h4['Word']),word_freq_df_p["Weight"]+0.5,word_freq_df_p["Weight"])
        indexNames = word_freq_df_p[word_freq_df_p["TotWeight"] > 0.5].index
        word_freq_df_p.drop(indexNames , inplace=True)

        #D6 -D7 comparison
        #add 0.4
        word_freq_df_h5["TotWeight"] = np.where(word_freq_df_h5['Word'].isin(word_freq_df_p['Word']),word_freq_df_h5["Weight"]+0.5,word_freq_df_h5["Weight"])
        word_freq_df_p["TotWeight"]  = np.where(word_freq_df_p['Word'].isin(word_freq_df_h5['Word']),word_freq_df_p["Weight"]+0.4,word_freq_df_p["Weight"])
        indexNames = word_freq_df_p[word_freq_df_p["TotWeight"] > 0.5].index
        word_freq_df_p.drop(indexNames , inplace=True)


        word_freq_df_Total     = word_freq_df_Title.append(word_freq_df_h1)
        word_freq_df_Total     = word_freq_df_Total.append(word_freq_df_h2)
        word_freq_df_Total     = word_freq_df_Total.append(word_freq_df_h3)
        word_freq_df_Total     = word_freq_df_Total.append(word_freq_df_h4)
        word_freq_df_Total     = word_freq_df_Total.append(word_freq_df_h5)

        #arrange in descending order
        word_freq_df_Totals    = pd.DataFrame(word_freq_df_Total, columns=['TotWeight']).sort_values('TotWeight',ascending=False)

        return word_freq_df_Totals

    # open text file
    # write keywords
    #close file
    file = open("webdirectory.txt","a")
    file.write(line+"\n")
    file.write(str(CalCulateTotal())+"\n")
    file.close()




print("url list write-Complete")

#write the list of common words
file = open("commonwords.txt","w")
file.write(str(STOPWORDS))
file.close()
print("common words write-Complete")

url list write-Begin
url list write-Complete
common words write-Complete
Wall time: 4min 10s
